# Part I. ETL Pipeline for Pre-Processing the Files

## Run the following code to pre-process the csv files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))

/Users/moe/Projects/data-engineering-nanodegree/1-data-modeling/p2-data-modeling-with-apache-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_full.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_full.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

## The newly create event_datafile_full.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should look like in the <font color=red>**event_datafile_full.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
import yaml
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

def create_cassandra_client_with_auth(configfilename):
    with open(configfilename, 'r') as f:
        config = yaml.load(f)['cassandra']
    
    auth_provider = PlainTextAuthProvider(
                username=config['username'],
                password=config['password'])
    
    cluster = Cluster(auth_provider=auth_provider)
    return cluster, cluster.connect()

# Change to the name of your secrets file. Never check into source control!
cluster, session = create_cassandra_client_with_auth('cassandra-secrets.yaml')

/Users/moe/Virtualenvs/p2-data-modeling-with-apache-cassandra/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


#### Create Keyspace


In [15]:
session.execute("create keyspace if not exists sparkify with replication = {'class': 'SimpleStrategy', 'replication_factor': 1};")

#### Set Keyspace


In [16]:
session.execute("use sparkify;")


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [17]:

## Create table to be able to answer queries for access pattern #1
session.execute("""create table if not exists songplays_by_sessionid 
                   (session_id int, item_in_session int, artist text, song_title text, song_length text, primary key(session_id, item_in_session) )""")

                    

In [18]:
# Load data into the table
file = 'event_datafile_full.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songplays_by_sessionid (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "values(%s, %s, %s, %s, %s)"
        
        # Todo: catch exceptions from attempting to cast strings to ints
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[5]))

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
rows = session.execute("""select artist,
                                 song_title,
                                 song_length
                   from sparkify.songplays_by_sessionid
                   where session_id = 338 and item_in_session = 4;""")
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [35]:
session.execute("""create table if not exists songplays_by_userid 
                   (user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, primary key((user_id, session_id), item_in_session) )""")

In [36]:
# Load data into the table
file = 'event_datafile_full.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songplays_by_userid (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        
        # Todo: catch exceptions from attempting to cast strings to ints
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
rows = session.execute("""select artist,
                                 song_title,
                                 first_name,
                                 last_name,
                                 item_in_session
                          from sparkify.songplays_by_userid
                          where user_id = 10 and session_id = 182;""")
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name, row.item_in_session)


Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


                    
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [32]:
session.execute("""create table if not exists songplays_by_song 
                   (user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, primary key(song_title, user_id) )""")


                    

In [33]:
file = 'event_datafile_full.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songplays_by_song (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        
        # Todo: catch exceptions from attempting to cast strings to ints
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [34]:
rows = session.execute("""select first_name,
                                 last_name
                          from sparkify.songplays_by_song
                          where song_title = 'All Hands Against His Own';""")
for row in rows:
    print(row.first_name, row.last_name)





Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop the tables before closing out the sessions

In [ ]:
# session.execute("drop table songplays_by_song;")
# session.execute("drop table songplays_by_userid;")
# session.execute("drop table songplays_by_sessionid;")

## Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()
